## Relevant Links
- Examples: https://docs.google.com/spreadsheets/d/14pdiODt9PmsD2F___Lpv2rO-RByTtC8PYcEfxL8iIuY/edit#gid=186467173
- LangChain: https://github.com/hwchase17/langchain
- Chroma: no docker or API_KEY - https://www.trychroma.com/
- OpenAI account usage: https://platform.openai.com/account/usage

## Tasks

- **Eric & Jacob**: Validation (`validate_stages.py` is not yet working, and has not been incorporated)
- **Jacob**: Label class recognition
    - determine what classes the user is referring to for each label field
- **Allen**: Evaluation Identification Stage:
    - `links/evaluation_run_selector.py` 
    - add stage to `dataset_view_generator.py`
    - add `prompts/evaluation_task_rules.txt` prompt
    - add examples to Examples spreadsheet in new tab, and then put in `examples` folder
- **Leila**: Add support for `hardness` brain runs
    - fill out the template in `links/brain_run_selector.py` 
    - add `prompts/hardness_task_rules.txt` prompt modeled after uniqueness and mistakenness.
    - new examples tab `hardness` modeled after `uniqueness` and `mistakenness` tabs and then put in `examples` folder
- **Vini**: More examples
    - scrape examples from community Slack covering as wide a range of scenarios as possible
    - complex filters and view expressions!
    - videos
    - every type of label
    - varied naming conventions

## Additional notes

- Want to validate embedded fields
- At present, there is no support for multi-line Python code to generate views. It all needs to be done inline
- No support for groups, 3D/point clouds, `concat()`, or `mongo()`.

## Getting started

`pip install openai langchain chroma`

Then create an OpenAI account and generate an API key

`export OPENAI_API_KEY=...`

In [ ]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

## Get similar prompts

In [ ]:
from links.view_stage_example_selector import generate_view_stage_examples_prompt

In [ ]:
query = "Five random images from the dataset"

In [ ]:
view_stage_examples_prompt = generate_view_stage_examples_prompt(dataset, query)

In [ ]:
print(view_stage_examples_prompt)

## Generate View Stage Descriptions Prompt

In [ ]:
from links.view_stage_description_selector import generate_view_stage_descriptions_prompt

In [ ]:
view_stage_descriptions_prompt = generate_view_stage_descriptions_prompt(view_stage_examples_prompt)

In [ ]:
print(view_stage_descriptions_prompt)

## Algorithm selector

In [18]:
from links.algorithm_selector import select_algorithms

In [19]:
query = "Five most unique images from the dataset"
select_algorithms(query)

['uniqueness']

In [3]:
query = "50 least unique images from the dataset that were hard"
select_algorithms(query)

['uniqueness', 'hardness']

In [4]:
query = "Five most mistaken images from the dataset"
select_algorithms(query)

['mistakenness']

In [5]:
query = "false positive predictions"
select_algorithms(query)

['evaluation']

## Run selector

In [1]:
from links.run_selector import select_runs

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F
dataset = foz.load_zoo_dataset("quickstart")

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
query = "most unique images from the dataset with key 'uniqueness1 that are similar to image 10"
select_runs(dataset, query, ["uniqueness", "image_similarity"])

{'uniqueness': 'uniqueness1', 'image_similarity': 'img_sim'}

In [15]:
query = "Five most mistaken images from the dataset with ground truth field 'predictions"
select_runs(dataset, query, ["mistakenness"])

{'mistakenness': 'mistakenness1'}

In [6]:
query = "Five most challenging images from dataset"
select_runs(dataset, query, ["hardness"])

{'hardness': 'hardness'}

In [ ]:
query = "Difficulty from key 'test_hardness'"
select_runs(dataset, query, ["hardness"])

In [4]:
query = "true positives"
select_runs(dataset, query, ["evaluation"])

{'evaluation': {'key': 'eval_cls',
  'method': 'simple',
  'pred_field': 'cls_pd',
  'gt_field': 'cls_gt'}}

In [3]:
query = "images with largest height"
select_runs(dataset, query, ["metadata"])

No metadata found. To compute metadata for your samples, please run the following command:

        ```
        dataset.compute_metadata()
        ```
        


{}

## Field selector

In [ ]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [ ]:
from links.field_selector import select_fields

In [ ]:
query = "sort pred1 by number of detections"
fields = select_fields(dataset, query)
print(fields)

## Label class selector

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

dataset = foz.load_zoo_dataset("quickstart")

In [ ]:
from links.label_class_selector import select_label_classes

In [ ]:
query = "sort Bear and glove predictions by number of detections"
prompt = select_label_classes(dataset, query, "[predictions]")
print(prompt)

## DatasetView generator

In [1]:
from gpt_view_generator import get_gpt_view_text

Using embedded DuckDB without persistence: data will be transient


In [2]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [10]:
query = "images with tables"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: images with tables
Identified likely view stages: ['match', 'sort_by_similarity', 'skip', 'exclude', 'exists', 'filter_labels', 'limit', 'select_fields', 'sort_by', 'take']
Identified algorithms: ['text_similarity']
No similarity index found that supports text prompts. To generate a similarity index for your samples, please run the following command:

        ```
        import fiftyone.brain as fob
        fob.compute_similarity(
            dataset, 
            model='clip-vit-base32-torch',
            brain_run_key='text_sim',
            )
        ```
        
Identified potentially relevant fields: [ground_truth]
Class name table not found for label ground_truth
Identified label classes: {'ground_truth': ['table']}
[match(
    F("ground_truth.detections.label").contains("table")
)]


In [ ]:
query = "high confidence detections"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

In [ ]:
query = "first 30 images with a ground truth detection"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

In [ ]:
query = "sort by image uniqueness1 for images with a >0.5 confidence classification"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

In [10]:
query = "find the images most resembling a farm scene"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: find the images most resembling a farm scene
Identified likely view stages: ['match', 'sort_by_similarity', 'skip', 'exclude', 'filter_labels', 'limit', 'select_fields', 'sort_by', 'take', 'exclude_by']
Identified algorithms: ['image_similarity']
Identified runs: {'image_similarity': {'key': 'clip', 'method': 'sklearn', 'embeddings_field': None, 'model': 'clip-vit-base32-torch', 'patches_field': None}}
Identified potentially relevant fields: [ground_truth, predictions]
Class name farm not found for label ground_truth
Identified label classes: {'ground_truth': ['farm'], 'predictions': []}
[sort_by_similarity("farm", brain_key="clip")]


In [9]:
query = "get true positives with eval_det_iou08_tp"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: get true positives with eval_det_iou08_tp
Identified likely view stages: ['exists', 'filter_labels', 'match', 'filter_field', 'filter_keypoints', 'match_labels', 'sort_by', 'exclude', 'exclude_by', 'exclude_fields']
Identified algorithms: ['evaluation']
Identified runs: {'evaluation': {'key': 'eval_cls', 'method': 'simple', 'pred_field': 'cls_pd', 'gt_field': 'cls_gt'}}
Identified potentially relevant fields: [eval_det_iou08_tp]
[filter_labels(
    "eval_det_iou08_tp",
    F("eval_det_iou08_tp") == "tp"
)]


In [3]:
query = "smallest 20 images"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: smallest 20 images
Getting or creating embeddings for queries...
Loading embeddings from file...
Saving embeddings to file...
Identified likely view stages: ['match', 'limit', 'sort_by_similarity', 'skip', 'exclude', 'sort_by', 'take', 'exclude_by', 'exclude_fields', 'exclude_frames']
Identified algorithms: ['metadata']
No metadata found. To compute metadata for your samples, please run the following command:

        ```
        dataset.compute_metadata()
        ```
        
field_names ['id', 'filepath', 'tags', 'metadata', 'ground_truth', 'uniqueness', 'predictions']
available_fields ['id: string', 'filepath: string', 'tags: list']
fs ['id', 'filepath', 'tags', 'metadata', 'ground_truth', 'uniqueness', 'predictions']
fn ground_truth
fn uniqueness
fn predictions
Identified potentially relevant fields: [filepath]
[sort_by("metadata.size_bytes"), limit(20)]


## Roadmap

### Validation and error handling
- even fully fleshed out, this is not going to be 100% accurate. Need to validate that it is actually creating a valid DataSetView
- if it isn't, prompt the user for more specific information
    - this could potentially be tailored to which part of the process it failed

### Plugin
- Everything will be wrapped in a single python function `generate_datasetview_with_chatgpt(dataset, prompt)`
- Then need to turn this into a plugin. It will take the `session.dataset` as dataset, and will set `session.view`
- This will probably be a menu-item plugin - we could use the ChatGPT symbol - it will take user input
- Would love to have a toggle the user can specify for whether they want 
    - the view created from scratch, or 
    - the view stages concatenated with their existing view

### Future
- Memory/chat history
- More general question-answering:
    - First stage: determine whether the user is asking a question about the entire dataset, or individual samples
    - Use ChatGPT as a dispatcher, deciding what other models/processes to invoke.
        - if it is about aggregations, then decide what FiftyOne aggregation to perform, and interpret the results
        - if it is about a single image, employ BLIPv2 or equivalent...